<a href="https://colab.research.google.com/github/yootaka/AiQuest/blob/main/en4_6keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical

from keras.layers import Dense
from keras.models import Sequential
from keras.models import load_model

import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Google ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
X = []
Y = []

folder_path = "/content/drive/My Drive/Colab Notebooks/AiQuest/en4/train/"
file_list = os.listdir(folder_path)

In [15]:
X_test = []
Y_test = []

folder_path_test = "/content/drive/My Drive/Colab Notebooks/AiQuest/en4/test/test/"
file_list_test = os.listdir(folder_path_test)

In [16]:
print(file_list) #ファイル確認

['bridge_000.jpeg', 'bridge_001.jpeg', 'bridge_003.jpeg', 'bridge_002.jpeg', 'bridge_005.jpeg', 'bridge_004.jpeg', 'bridge_006.jpeg', 'bridge_007.jpeg', 'bridge_009.jpeg', 'bridge_008.jpeg', 'bridge_011.jpeg', 'bridge_010.jpeg', 'bridge_013.jpeg', 'bridge_012.jpeg', 'bridge_014.jpeg', 'bridge_015.jpeg', 'bridge_016.jpeg', 'bridge_017.jpeg', 'bridge_018.jpeg', 'bridge_019.jpeg', 'bridge_020.jpeg', 'bridge_021.jpeg', 'bridge_022.jpeg', 'horn_000.jpeg', 'horn_001.jpeg', 'horn_002.jpeg', 'horn_003.jpeg', 'horn_006.jpeg', 'horn_005.jpeg', 'horn_004.jpeg', 'horn_007.jpeg', 'horn_008.jpeg', 'horn_009.jpeg', 'horn_010.jpeg', 'horn_011.jpeg', 'horn_013.jpeg', 'horn_012.jpeg', 'horn_014.jpeg', 'horn_015.jpeg', 'horn_017.jpeg', 'horn_016.jpeg', 'horn_018.jpeg', 'horn_019.jpeg', 'horn_020.jpeg', 'horn_022.jpeg', 'horn_021.jpeg', 'horn_023.jpeg', 'horn_025.jpeg', 'horn_024.jpeg', 'horn_026.jpeg', 'horn_027.jpeg', 'horn_028.jpeg', 'horn_029.jpeg', 'horn_030.jpeg', 'horn_031.jpeg', 'horn_033.jpeg', '

In [17]:
print(file_list_test) #ファイル確認

['120.jpeg', '197.jpeg', '053.jpeg', '137.jpeg', '051.jpeg', '062.jpeg', '173.jpeg', '059.jpeg', '177.jpeg', '128.jpeg', '203.jpeg', '187.jpeg', '029.jpeg', '151.jpeg', '006.jpeg', '068.jpeg', '150.jpeg', '113.jpeg', '202.jpeg', '211.jpeg', '191.jpeg', '025.jpeg', '033.jpeg', '176.jpeg', '109.jpeg', '139.jpeg', '037.jpeg', '103.jpeg', '045.jpeg', '091.jpeg', '024.jpeg', '146.jpeg', '088.jpeg', '110.jpeg', '101.jpeg', '081.jpeg', '072.jpeg', '013.jpeg', '094.jpeg', '115.jpeg', '212.jpeg', '189.jpeg', '038.jpeg', '004.jpeg', '061.jpeg', '055.jpeg', '162.jpeg', '052.jpeg', '122.jpeg', '111.jpeg', '102.jpeg', '116.jpeg', '209.jpeg', '083.jpeg', '049.jpeg', '074.jpeg', '080.jpeg', '160.jpeg', '188.jpeg', '071.jpeg', '067.jpeg', '123.jpeg', '148.jpeg', '100.jpeg', '075.jpeg', '205.jpeg', '035.jpeg', '011.jpeg', '019.jpeg', '093.jpeg', '182.jpeg', '012.jpeg', '200.jpeg', '106.jpeg', '207.jpeg', '028.jpeg', '079.jpeg', '170.jpeg', '125.jpeg', '031.jpeg', '086.jpeg', '172.jpeg', '147.jpeg', '15

In [22]:
image_size = 256

for file in file_list:
    try:
        image = Image.open(folder_path + file)
    except:
        print('error', file)
        continue
    
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    X.append(data)
    if 'bridge' or 'horn' or 'potato' in file:
        Y.append(1) #不良品
    else:
         Y.append(0) #良品

X_train = np.array(X) #画像データ
y_train = np.array(Y) #ラベルデータ

In [27]:
print(X_train.shape)
print(y_train.shape)

(833, 256, 256, 3)
(833,)


In [34]:
image_size = 256

for file in file_list_test:
    try:
        image = Image.open(folder_path_test + file)
    except:
        print('error', file)
        continue
    
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    X.append(data)
    Y.append(0) #初期値

X_test = np.array(X) #画像データ
y_test = np.array(Y) #ラベルデータ

In [35]:
print(X_test.shape)
print(y_test.shape)

(1260, 256, 256, 3)
(1259,)


データ整形

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [28]:
# 画像を1次元配列にreshape
X_train = X_train.reshape(-1, image_size * image_size *3)
X_test = X_test.reshape(-1, image_size * image_size *3)

In [29]:

# 輝度値を0 ~ 1に入るように正規化
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [30]:
# one hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

モデル作成

In [31]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=image_size * image_size *3))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                12582976  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 12,583,106
Trainable params: 12,583,106
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=20)

Epoch 1/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0169 - accuracy: 0.9868
Epoch 2/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/20
42/42 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/20
42/42 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/20
42/42 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/20
42/42 [==============================] - 0s 10ms/step - loss: 0.0

In [33]:
model.evaluate(X_test, y_test)

ValueError: ignored

In [ ]:
model.save('model.h5')

In [ ]:
y_test